In [1]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [5]:
EXPERIMENT_GROUP = "exp-1-dataset-split-strategy"
BASELINE_SETUP = "random"
AVERAGE_EPOCHS = 0 # 5 or 0 | 0 for no averaging, must be odd
HAS_WEIGHTED_ACCURACY = False
FIRST_N_METRICS = 6 #6 # Only weighted_accuracy and macro_f1 for the results table in experiment results section
EXCLUDED_SETUPS = [] # [0, 1] | list of setups to exclude, by their index in the list of setups

In [3]:
def get_best_epoch_metrics(run, metric_name, average_epochs=0):
    history = pd.DataFrame(list(run.scan_history()))
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    if average_epochs > 0:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the average_epochs epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean() # TOOD: Fix error
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=0, has_weighted_accuracy=True):
    api = wandb.Api(timeout=60)

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        if run.state != 'finished':
            print(f"Run {run.url} - Skipping (not finished, state: {run.state})")
            continue
        if 'name' not in run.config:
            if 'experiment_name' in run.config:
                setup_name = run.config['experiment_name'].split('-')[-1]
                print(f"Run {run.url} - Missing 'name' in config, getting it from 'experiment_name' instead: {setup_name}")
            else:
                print(f"Run {run.url} - Missing 'name' in config, skipping")
                continue
        else:
            setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority' if has_weighted_accuracy else 'val_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority' if has_weighted_accuracy else 'val_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority' if has_weighted_accuracy else 'val_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS, has_weighted_accuracy=HAS_WEIGHTED_ACCURACY)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 22 runs for experiment group 'exp-1-dataset-split-strategy'
Run https://wandb.ai/crop-classification/messis/runs/tmmtb4ce - Missing 'name' in config, getting it from 'experiment_name' instead: random
Run https://wandb.ai/crop-classification/messis/runs/04lknbug - Missing 'name' in config, getting it from 'experiment_name' instead: random
Setup 'stratified-longer' - 1 runs
Setup 'stratified-long' - 1 runs
Setup 'sechidis' - 5 runs
Setup 'random' - 5 runs
Setup 'bands' - 5 runs
Setup 'stratified' - 5 runs
Run https://wandb.ai/crop-classification/messis/runs/islwn1g7 - Best epoch: 7434 - val_f1_tier3_majority: 0.194545179605484
Run https://wandb.ai/crop-classification/messis/runs/1wx9c8fy - Best epoch: 2064 - val_f1_tier3_majority: 0.14845073223114014
Run https://wandb.ai/crop-classification/messis/runs/lfj8cwu4 - Best epoch: 1050 - val_f1_tier3_majority: 0.10101296752691269
Run https://wandb.ai/crop-classification/messis/runs/hmm1uo25 - Best epoch: 1021 - val_f1_tier3_majority: 0.0

,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,stratified-longer,1,0,[islwn1g7],[7434],0.390121,0.000000,0.274251,0.000000,0.194545,...,0.260451,0.000000,0.185722,0.000000,0.868168,0.000000,0.725088,0.000000,0.679943,0.000000
1,stratified-long,1,0,[1wx9c8fy],[2064],0.389849,0.000000,0.268020,0.000000,0.148451,...,0.250327,0.000000,0.149984,0.000000,0.847390,0.000000,0.709557,0.000000,0.656246,0.000000
2,sechidis,5,0,"[lfj8cwu4, hmm1uo25, a42lq9q5, kstftel1, eyy30...","[1050, 1021, 1122, 1103, 1109]",0.397259,0.046656,0.227932,0.017764,0.104021,...,0.229077,0.018933,0.111453,0.010005,0.823467,0.030677,0.652736,0.013962,0.580093,0.011586
3,random,5,0,"[tmmtb4ce, 04lknbug, qj4o4ati, 2zgnr3fw, mudkt...","[1187, 1498, 1326, 1247, 1065]",0.428437,0.073525,0.234024,0.023781,0.114516,...,0.241598,0.024813,0.123559,0.015195,0.842893,0.011668,0.675422,0.012982,0.604248,0.004763
4,bands,5,0,"[fyumjoqi, 4nu14ssi, lzkvcm2x, mc68cdlz, blfh8...","[740, 1392, 1520, 899, 869]",0.397116,0.071495,0.222217,0.034746,0.107338,...,0.230282,0.025620,0.116377,0.017932,0.786599,0.019260,0.615557,0.056843,0.553128,0.057338
5,stratified,5,0,"[av80hhcp, lf9khgx7, 8591uumk, f8oi85s8, vjyei...","[1119, 1073, 1056, 1201, 816]",0.387703,0.040898,0.218297,0.015290,0.100110,...,0.218198,0.011233,0.103648,0.003964,0.797149,0.035573,0.628228,0.020299,0.560959,0.015068


In [6]:
# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
}

if HAS_WEIGHTED_ACCURACY:
    metric_name_mapping['val_weighted_accuracy_tier1_majority'] = 'W. Acc. $T_1$'
    metric_name_mapping['val_weighted_accuracy_tier2_majority'] = 'W. Acc. $T_2$'
    metric_name_mapping['val_weighted_accuracy_tier3_majority'] = 'W. Acc. $T_3$'
else:
    metric_name_mapping['val_accuracy_tier1_majority'] = 'Acc. $T_1$'
    metric_name_mapping['val_accuracy_tier2_majority'] = 'Acc. $T_2$'
    metric_name_mapping['val_accuracy_tier3_majority'] = 'Acc. $T_3$'

metric_name_mapping = { # Adding further metrics down here, to make sure we keep the most relevant metrics at the top
    **metric_name_mapping,
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

# Function to create a LaTeX table with bold best scores and underlined second-best scores
def create_latex_table(df, baseline_setup, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best and second-best scores for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    second_best_scores = {
        metric: sorted(df[f'{metric}_mean'].unique(), reverse=True)[1] if len(df[f'{metric}_mean'].unique()) > 1 else None
        for metric in metrics
    }

    # Reorder the dataframe to have the baseline setup first
    df = df.set_index('setup')
    if baseline_setup in df.index:
        df = df.loc[[baseline_setup] + [idx for idx in df.index if idx != baseline_setup]].reset_index()
    else:
        df = df.reset_index()

    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header
    headers = ['Metric'] + [f"{'$\\dagger$ ' if row['setup'] == baseline_setup else ''}{setup} ({row['num_runs']}-fold)" 
                            for idx, row in df.iterrows() for setup in [row['setup']]]
    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            elif second_best_scores[metric] and mean == second_best_scores[metric] * 100:
                value = f'\\underline{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0] 
    caption_averaged =  "" if average_epochs_used == 0 else f" averaged over {average_epochs_used} epochs"
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase{caption_averaged}, with mean and std reported. Best scores bold, second best underlined, $\\dagger$ is baseline.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")
df = df.sort_values(by='setup')

# Optional: Drop rows to exclude some setups
df = df.drop(EXCLUDED_SETUPS)

latex_table = create_latex_table(df, baseline_setup=BASELINE_SETUP, first_n_metrics=FIRST_N_METRICS)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcccccc}
			\toprule
			Metric & $\dagger$ random (5-fold) & bands (5-fold) & sechidis (5-fold) & stratified (5-fold) & stratified-long (1-fold) & stratified-longer (1-fold) \\
			\midrule
			F1 $T_1$ & \textbf{42.8\% ± 7.4\%} & 39.7\% ± 7.1\% & \underline{39.7\% ± 4.7\%} & 38.8\% ± 4.1\% & 39.0\% ± 0.0\% & 39.0\% ± 0.0\% \\
			F1 $T_2$ & 23.4\% ± 2.4\% & 22.2\% ± 3.5\% & 22.8\% ± 1.8\% & 21.8\% ± 1.5\% & \underline{26.8\% ± 0.0\%} & \textbf{27.4\% ± 0.0\%} \\
			F1 $T_3$ & 11.5\% ± 1.6\% & 10.7\% ± 2.3\% & 10.4\% ± 0.9\% & 10.0\% ± 0.4\% & \underline{14.8\% ± 0.0\%} & \textbf{19.5\% ± 0.0\%} \\
			Acc. $T_1$ & \textbf{43.2\% ± 7.7\%} & \underline{40.1\% ± 7.1\%} & 38.8\% ± 4.2\% & 38.3\% ± 3.4\% & 36.3\% ± 0.0\% & 36.5\% ± 0.0\% \\
			Acc. $T_2$ & 24.2\% ± 2.5\% & 23.0\% ± 2.6\% & 22.9\% ± 1.9\% & 21.8\% ± 1.1\% & \underline{25.0\% ± 0.0\%} & \textbf{26.0\% ± 0.0\%} \\
			Acc. $T_3$ & 12.4\% ± 1.5\% & 11.6\% 